In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(200_000_000_000)
 
import geopandas as gpd
import pandas as pd

from dask import delayed, compute
from dask.delayed import Delayed # use this for type hint
from siuba import *

import gtfs_utils_v2
from shared_utils import geography_utils, rt_dates, utils

analysis_date = rt_dates.DATES["oct2022"]

DASK_GCS = "gs://calitp-analytics-data/data-analyses/dask_test/v2_schedule/"

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
'''
daily_feeds_orgs = gtfs_utils_v2.daily_feed_to_organization(
    selected_date = analysis_date,
    get_df = True
)

daily_feeds_orgs.to_parquet("./data/daily_feeds_orgs.parquet")
'''

daily_feeds_orgs = pd.read_parquet("./data/daily_feeds_orgs.parquet")

In [3]:
test_operators = [
    "LA Metro Bus Schedule",
    "LA Metro Rail Schedule"
]

#daily_feeds_orgs[daily_feeds_orgs.name.isin(test_operators)]

test_feed_keys = daily_feeds_orgs[
    daily_feeds_orgs.name.isin(test_operators)].feed_key.tolist()

print(test_feed_keys)

['285e72c8fed97bcd0acb05aeda8aa985', 'bbba508561429fb97adb16f47bb4bdb1']


In [ ]:
def compute_delayed(
    feed_key: str, analysis_date: str, 
    trips: Delayed, route_info: Delayed, route_shapes: Delayed
):
    """
    Unpack the result from the dask delayed function. 
    The delayed object is a tuple, and since we just have 1 object, use 
    [0] to grab it.
    
    Export the pd.DataFrame or gpd.GeoDataFrame to GCS.
    
    https://stackoverflow.com/questions/44602766/unpacking-result-of-delayed-function
    """
    trips = compute(trips)[0]
    print("Computed trips")
    
    route_info = compute(route_info)[0]
    print("Computed route_info")

    route_shapes = compute(route_shapes)[0]
    print("Computed route_shapes")
    
    DATA_PATH = "./v2_data/"
    
    if not ((trips.empty) and (route_info.empty) and (route_shapes.empty)):
        trips.to_parquet(
            f"{DATA_PATH}trips_{feed_key}_{analysis_date}.parquet")
        print("Exported trips")
        
        route_info.to_parquet(
            f"{DATA_PATH}route_info_{feed_key}_{analysis_date}.parquet")
        print("Exported route_info")
        
        route_shapes.to_parquet(
            f"{DATA_PATH}route_shapes_{feed_key}_{analysis_date}.parquet")
        print("Exported route_shapes")


In [ ]:
for feed in test_feed_keys:
    trips = delayed(
        gtfs_utils_v2.get_trips(
            selected_date = analysis_date,
            subset_feeds = [feed],
            get_df = True
        )
    )
    
    route_info = delayed(
        gtfs_utils_v2.get_route_info(
            selected_date = analysis_date,
            subset_feeds = [feed],
            get_df = True
        )
    )
    
    route_shapes = delayed(
        gtfs_utils_v2.get_route_shapes(
            selected_date = analysis_date,
            subset_feeds = [feed],
            get_df = True,
        )
    )

    compute_delayed(feed, analysis_date, 
                    trips, route_info, route_shapes)